In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

In [2]:
sql = '''
SELECT *
FROM dividends
LIMIT 1'''
struct = pd.read_sql(sql, conpf)
struct.dtypes

id            int64
stock_id      int64
name         object
year          int64
quarter       int64
number        int64
ppu         float64
amt         float64
net         float64
x_date       object
p_date       object
dtype: object

In [3]:
sql = '''
SELECT YEAR(p_date) AS p_year, MONTH(p_date) AS p_month, name, year, quarter, 
number, ppu, amt, net, x_date, p_date
FROM dividends
'''
df = pd.read_sql(sql, conpf)
df.dtypes

p_year       int64
p_month      int64
name        object
year         int64
quarter      int64
number       int64
ppu        float64
amt        float64
net        float64
x_date      object
p_date      object
dtype: object

In [4]:
df['x_date'] = pd.to_datetime(df['x_date'])
df['p_date'] = pd.to_datetime(df['p_date'])
df.dtypes

p_year              int64
p_month             int64
name               object
year                int64
quarter             int64
number              int64
ppu               float64
amt               float64
net               float64
x_date     datetime64[ns]
p_date     datetime64[ns]
dtype: object

In [5]:
df_groupby_year = df.groupby('p_year')
type(df_groupby_year)

pandas.core.groupby.generic.DataFrameGroupBy

In [6]:
df_groupby_year.ngroups

6

In [7]:
df_groupby_year.size()

p_year
2017     30
2018     48
2019     51
2020     54
2021    106
2022     71
dtype: int64

In [8]:
df_cur_yr = df_groupby_year.get_group(2022)
df_cur_yr.columns

Index(['p_year', 'p_month', 'name', 'year', 'quarter', 'number', 'ppu', 'amt',
       'net', 'x_date', 'p_date'],
      dtype='object')

In [9]:
output = df_cur_yr.groupby('p_month').net.sum()
output

p_month
2       1547.10
3      55572.24
4      44670.00
5     214884.69
6      51232.10
8       7155.00
9     164892.61
10      5265.00
12    115868.30
Name: net, dtype: float64

In [10]:
data_path = "../data/"
file_name = 'tmp-file-of-dividends.csv'
data_file = data_path + file_name
output.to_csv(data_file, index=True)

In [11]:
df_cur_yr.net.sum()

661087.04

### Old method

In [12]:
mask = df.p_year == 2022

In [13]:
ttl_by_month = df[mask].groupby(['p_year','p_month','name'], as_index=True).agg(
    {
        'net':['sum','count'],
    }
)
ttl_by_month

net      
                           sum count
p_year p_month name                 
2022   2       LPF      1547.1     1
       3       AIMIRT   1989.0     1
               CPNCG    1999.8     1
               DIF     15660.0     1
               JASIF   27500.0     1
...                        ...   ...
       12      JASIF   29900.0     1
               RCL     49800.0     1
               TFFIF    1003.0     1
               WHAIR    6818.4     1
               WHART    6893.1     1

[71 rows x 2 columns]

In [14]:
ttl_by_month.groupby(level='p_month').sum()

net      
               sum count
p_month                 
2          1547.10     1
3         55572.24     6
4         44670.00     6
5        214884.69    19
6         51232.10     5
8          7155.00     2
9        164892.61    21
10         5265.00     1
12       115868.30    10

In [15]:
ttl_by_month.sum()

net  sum      661087.04
     count        71.00
dtype: float64

In [16]:
ttl_by_month.sort_values(['p_month','name'],ascending=[True,True])

net      
                           sum count
p_year p_month name                 
2022   2       LPF      1547.1     1
       3       AIMIRT   1989.0     1
               CPNCG    1999.8     1
               DIF     15660.0     1
               JASIF   27500.0     1
...                        ...   ...
       12      JASIF   29900.0     1
               RCL     49800.0     1
               TFFIF    1003.0     1
               WHAIR    6818.4     1
               WHART    6893.1     1

[71 rows x 2 columns]

In [17]:
ttl_by_month['net'].sum()

sum      661087.04
count        71.00
dtype: float64

In [18]:
sql = '''
SELECT YEAR(p_date) AS pay_year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
GROUP BY pay_year
ORDER BY pay_year DESC'''
ttl_by_year = pd.read_sql(sql, conpf)
ttl_by_year

,pay_year,grs_amt,net_amt,refund
0,2022,708367.91,661087.04,47280.87
1,2021,752315.10,700874.50,51440.60
2,2020,616228.94,576030.84,40198.10
3,2019,552424.34,510266.46,42157.88
4,2018,351895.60,309725.58,42170.02
5,2017,245981.10,219514.23,26466.87


In [19]:
sql = '''
SELECT name, COUNT(*) AS qtrs, SUM(net) AS ttl_net
FROM dividends
GROUP BY name
ORDER BY SUM(net) DESC'''
total = pd.read_sql(sql, conpf)
total.sample(10)

,name,qtrs,ttl_net
0,JASIF,19,438400.00
31,PTTEP,4,19485.00
7,SAT,5,96040.58
94,GL,1,2466.00
96,WHA,1,2407.50
27,DTAC,2,22842.00
91,VIBHA,1,2916.00
75,PDG,1,4950.00
108,VNG,1,1500.00
38,CPNCG,4,12993.30


### Summarized profit by name

In [ ]:
sql = '''
SELECT name, sum(profit) AS profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
GROUP BY stocks.name
ORDER BY sum(profit) DESC LIMIT 10'''
profits_by_stock = pd.read_sql(sql, conpf)
profits_by_stock

In [ ]:
sql = '''
SELECT YEAR(date) AS year, MONTH(date) AS month, profit
FROM sells 
WHERE YEAR(date) = 2022
ORDER BY YEAR(date) DESC, MONTH(date) DESC
'''
sells = pd.read_sql(sql, conpf)
sells

In [ ]:
grouped = sells.groupby(['year','month'])
grouped

In [ ]:
profit_by_month = grouped['profit'].sum()
profit_by_month

In [ ]:
grouped.agg(['sum','mean', 'max', 'min','count'])

### Temporary process to calculate dividend portion of profit

In [ ]:
year = 2022
quarter = 4

In [ ]:
sql = """
SELECT name, year, quarter, aq_eps, ay_eps
FROM epss 
WHERE year = %s AND quarter = %s
"""
sql = sql % (year, quarter)
df_epss = pd.read_sql(sql, conlt)
df_epss.head()

In [ ]:
sql = '''
SELECT name, dividend
FROM dividend 
'''
df_dividend = pd.read_sql(sql, const)
df_dividend

In [ ]:
df_merge = pd.merge(df_epss, df_dividend, on='name', how='inner')
df_merge['xxx_pct'] = df_merge.dividend / df_merge.aq_eps * 100
df_merge.set_index('name',inplace=True)
df_merge.sort_values(['xxx_pct'],ascending=[True])

In [ ]:
df_merge.loc['TFFIF']